### Create an OpenRecon Docker image and .zip package
*&copy; Siemens Healthineers AG - All Rights Reserved*<br>
*Restricted - Unauthorized copying of this file, via any medium is strictly prohibited*<br>
<sub>(Version Mar 11th 2024)</sub><br><br>
The `OpenReconLabel.json` file contains metadata describing the configuration of the app to OpenRecon.  It is [base64-encoded](https://en.wikipedia.org/wiki/Base64) and stored in the Docker image metadata label `com.siemens-healthineers.magneticresonance.openrecon.metadata:1.1.0`.

Edit `OpenReconLabel.json` and the `baseDockerImage` below, then run the cell to validate its contents and create a `Dockerfile` which attaches the OpenRecon metadata.

In [13]:
import json
import jsonschema
import base64
import os
# Validate JSON file against OpenRecon schema and write Dockerfile
jsonFilePath    = 'openrecon_json_ui.json' # needs updating
schemaFilePath  = 'OpenReconSchema_1.1.0.json' # stays the same
dockerfilePath = '../OpenRecon.dockerfile' # changes automatically
baseDockerImage = 'kspacekelvin/fire-python' # stays the same - though we're using nvidia/cuda now
docker_build_context = '../'
#    conda run -n yuccaenv conda install -c pytorch fastai=1.0.61 Pillow beautifulsoup4 bottleneck dataclasses fastprogress=0.2.1 matplotlib numexpr "numpy>=1.15" packaging pandas pyyaml requests scipy && \
#  openrecon_ichilovsagollab_fetalbrainsegmentation:v1.1.0
def validateJson(jsonFilePath, schemaFilePath):
    try:
        # Load the JSON data from the file
        with open(jsonFilePath, 'r') as jsonFile:
            jsonData = json.load(jsonFile)

        # Load the JSON schema from the file
        with open(schemaFilePath, 'r') as schemaFile:
            schemaData = json.load(schemaFile)

        # Create a JSON Schema validator
        validator = jsonschema.Draft7Validator(schemaData)

        # Validate the JSON data against the schema
        errors = list(validator.iter_errors(jsonData))

        if not errors:
            print("JSON is valid against the schema.")
            return True
        else:
            print("JSON is not valid against the schema. Errors:")
            for error in errors:
                print(error)
            return False
    
    except Exception as e:
        print(f"An error occurred: {e}")

# === Write Dockerfile ===
if validateJson(jsonFilePath, schemaFilePath):
    with open(jsonFilePath, 'r') as jsonFile:
        jsonData = json.load(jsonFile)

    encoded = base64.b64encode(json.dumps(jsonData).encode("utf-8")).decode("utf-8")
    labelLine = f'LABEL "com.siemens-healthineers.magneticresonance.openrecon.metadata:1.1.0"="{encoded}"'

    with open(dockerfilePath, 'w') as f:
        f.write(f'''# ------------------------------------------------------------
#  OpenRecon Fetal Brain Segmentation Pipeline
#  Multi-stage build: ISMRMRD + Fetal Brain Pipeline
# ------------------------------------------------------------

# === Stage 1: Build ISMRMRD ===
FROM nvidia/cuda:11.0.3-cudnn8-devel-ubuntu18.04 as mrd_converter

WORKDIR /tmp

# Install build dependencies
RUN apt-get update && \\
    apt-get install -y --no-install-recommends \\
        build-essential \\
        cmake \\
        git \\
        libhdf5-dev \\
        libxml2-dev \\
        libxslt1-dev \\
        libboost-all-dev \\
        libfftw3-dev \\
        pkg-config \\
        wget && \\
    rm -rf /var/lib/apt/lists/*

# Build PugiXML from source
RUN git clone https://github.com/zeux/pugixml.git && \\
    cd pugixml && \\
    mkdir build && cd build && \\
    cmake .. && \\
    make -j$(nproc) && \\
    make install

# Build ISMRMRD from source
RUN git clone https://github.com/ismrmrd/ismrmrd.git && \\
    cd ismrmrd && \\
    mkdir build && cd build && \\
    cmake -DBUILD_UTILITIES=OFF -DBUILD_EXAMPLES=OFF -DBUILD_TESTS=OFF .. && \\
    make -j$(nproc) && \\
    make install

# === Stage 2: Runtime with Fetal Brain Pipeline ===
FROM nvidia/cuda:11.0.3-cudnn8-runtime-ubuntu18.04

# === OpenRecon UI Label ===
{labelLine}

WORKDIR /workspace
ENV DEBIAN_FRONTEND=noninteractive
ENV PIP_PREFER_BINARY=1

# Copy ISMRMRD libraries and development files from build stage
COPY --from=mrd_converter /usr/local/lib/libismrmrd* /usr/local/lib/
COPY --from=mrd_converter /usr/local/include/ismrmrd /usr/local/include/ismrmrd
COPY --from=mrd_converter /usr/local/lib/libpugixml* /usr/local/lib/
COPY --from=mrd_converter /usr/local/include/pugixml* /usr/local/include/
COPY --from=mrd_converter /usr/local/lib/pkgconfig/ /usr/local/lib/pkgconfig/
COPY --from=mrd_converter /usr/local/share/ismrmrd/ /usr/local/share/ismrmrd/

# Install runtime dependencies and build tools for Python packages
RUN apt-get update && \\
    apt-get install -y --no-install-recommends \\
        wget \\
        git \\
        libgl1 \\
        libglib2.0-0 \\
        python3.8 \\
        python3.8-venv \\
        python3.8-dev \\
        python3-pip \\
        build-essential \\
        cmake \\
        libhdf5-100 \\
        libhdf5-dev \\
        libxml2 \\
        libxml2-dev \\
        libxslt1.1 \\
        libxslt1-dev \\
        libboost-program-options1.65.1 \\
        libboost-system1.65.1 \\
        libboost-filesystem1.65.1 \\
        libboost-thread1.65.1 \\
        libboost-dev \\
        libfftw3-3 \\
        libfftw3-dev \\
        pkg-config && \\
    rm -rf /var/lib/apt/lists/*

# Update library cache
RUN ldconfig

# Make python3.8 the default "python"
RUN update-alternatives --install /usr/bin/python python /usr/bin/python3.8 1 && \\
    update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1

# Upgrade pip, setuptools, wheel to latest
RUN python -m pip install --no-cache-dir --upgrade pip setuptools wheel

# Install numpy first (required by ismrmrd-python-tools)
RUN python -m pip install --no-cache-dir numpy

# Install ISMRMRD Python libraries (now that C++ libs are available)
RUN python -m pip install --no-cache-dir ismrmrd-python-tools

# Install SimpleITK *only* from a prebuilt wheel
RUN python -m pip install --no-cache-dir --timeout=300 --retries=3 --only-binary=SimpleITK SimpleITK==2.3.1

# Copy fetal brain pipeline requirements and install with compatibility fixes
COPY fetal-brain-measurement/requirements.txt .
RUN grep -v '^SimpleITK' requirements.txt > reqs_original.txt && \\
    sed 's/numpy==1.14.5/numpy==1.21.0/g' reqs_original.txt | \\
    sed 's/tensorflow-gpu==1.14.0/tensorflow==2.8.0/g' | \\
    sed 's/Bottleneck==1.3.1/Bottleneck==1.3.5/g' | \\
    sed 's/mkl-random==1.1.1/mkl-random==1.2.2/g' | \\
    grep -v '^h5py\\|^matplotlib\\|^spacy\\|^en_core_web_sm\\|^jupyter\\|^notebook\\|^dataclasses\\|^contextvars\\|^mkl-\\|^tensorflow==' > reqs.txt && \\
    python -m pip install --no-cache-dir --timeout=300 -r reqs.txt

# Copy fetal brain pipeline code
COPY fetal-brain-measurement/ /workspace/fetal-brain-measurement/

# Copy OpenRecon server code
COPY python-ismrmrd-server/ /opt/code/python-ismrmrd-server/

# Copy OpenRecon handler files to server directory
COPY fetal-brain-measurement/openrecon.py /opt/code/python-ismrmrd-server/
COPY fetal-brain-measurement/openrecon.json /opt/code/python-ismrmrd-server/

# Set PYTHONPATH for fetal brain pipeline and OpenRecon
ENV PYTHONPATH="/workspace/fetal-brain-measurement:/workspace/fetal-brain-measurement/Code/FetalMeasurements-master/SubSegmentation:/opt/code/python-ismrmrd-server"

# Set working directory to OpenRecon server
WORKDIR /opt/code/python-ismrmrd-server

# Default command to run OpenRecon server with fetal brain handler
CMD ["python", "main.py", "-d=openrecon"]
''')
        
    print("Dockerfile written:", os.path.abspath(dockerfilePath))
else:
    raise Exception("Not writing Dockerfile: JSON is invalid.")

JSON is valid against the schema.
Dockerfile written: c:\OpenRecon_IchilovSagolLab_FetalBrainSegmentation_V1.0.0\OpenRecon.dockerfile


The Docker image can be built with the above `Dockerfile`.  For deployment on to a scanner, a saved copy of the Docker image (.tar) is bundled into a .zip file along with a documentation .pdf.  These three files must have the same name, follow the format `OpenRecon_<Vendor>_<Name>_V<version>`, and be consistent with the information in the JSON metadata.

**Note**: The zip file must be created using the [Deflate compression algorithm instead of Deflate64](https://en.wikipedia.org/wiki/ZIP_(file_format)#Compression_methods).  If using Windows, right-clicking on files and selecting "Send to -> Compressed (zipped) folder" [will use Deflate64 for a total file size >2GB](https://learn.microsoft.com/en-us/answers/questions/969084/how-does-windows-decide-whether-to-use-deflate-or) and will thus be incompatible.  Creating a zip file with [7-Zip](https://www.7-zip.org/) will use the Deflate algorithm for all file sizes.

Edit the `docsFile` variable and run the cell below.  It will assemble the file name based on the JSON metadata from the previous step, build the Docker image, save a .tar file, package it along with the docs .pdf into a .zip.

In [14]:
# Build Docker image, save to a .tar file, and package into a .zip file for OpenRecon
# The documentation must be a valid PDF!
docsFile = 'C:/OpenRecon_IchilovSagolLab_FetalBrainSegmentation_V1.0.0/additional-readme/openrecon_README[1].pdf' #'OpenRecon_DemoCorp_InvertContrast_V1.3.0.pdf' #does not actually need to have the correct name!

# Filename must match information contained in the JSON
version = jsonData['general']['version']
vendor  = jsonData['general']['vendor']
name    = jsonData['general']['name']['en']

# remove special characters, spaces, and underscores from name and vendor name for docker image name and base filename
import re
vendor = re.sub(r'[\W_]+', '', vendor)
name   = re.sub(r'[\W_]+', '', name)

# Check documentation file exists
if not os.path.isfile(docsFile):
    raise Exception('Could not find documentation file: ' + docsFile)

# Check 7-zip exists
zipExe = "C:/Program Files/7-Zip/7z.exe"     #'C:\\Program Files\\7-Zip\\7z.exe'
if not os.path.isfile(zipExe):
    raise Exception('Could not find 7-Zip executable: ' + docsFile + '\nPlease download and install 7-Zip')

dockerImagename = ('OpenRecon_' + vendor + '_' + name + ':' +  'V' + version).lower()
baseFilename    =  'OpenRecon_' + vendor + '_' + name +       '_V' + version

# === Create output folder ===
output_dir = os.path.join("..", "results", baseFilename)
os.makedirs(output_dir, exist_ok=True)

from checkDockerVersion import checkDockerVersion
checkDockerVersion("25.0.0")

import subprocess
import shutil

try:
    # Build Docker image
    print('Attempting to create Docker image with tag:', dockerImagename, '...') #'--progress=plain', , shell=True
    output = subprocess.check_output(
                                        [
                                            'docker', 'build',
                                            '--platform=linux/amd64',
                                            '--no-cache',
                                            '-t', dockerImagename,
                                            '-f', dockerfilePath,
                                            docker_build_context
                                        ],
                                        stderr=subprocess.STDOUT,
                                        text=True
                                    )
    
    print('Docker build output:\n' + output)

    # Save Docker image to a .tar file
    print('Saving Docker image file with name:', baseFilename + '.tar', '...')
    tar_path = os.path.join(output_dir, baseFilename + '.tar') # this or  baseFilename + '.tar',
    output = subprocess.check_output(                            #, shell=True
                                        [
                                            'docker', 'save',
                                            '-o',tar_path, dockerImagename
                                        ],
                                        stderr=subprocess.STDOUT,
                                        text=True
                                    ) 
    
    print('Docker save output:\n' + output) 

    # Copy documentation file with appropriate filename
    print('Copying documentation to file with name:', baseFilename + '.pdf', '...')
    try:
        pdf_path = os.path.join(output_dir, baseFilename + '.pdf')
        shutil.copy(docsFile, pdf_path)
        print(f'File copied from {docsFile} to {baseFilename}.pdf')
    except IOError as e:
        print(f'An error occurred: {e}')

    # Zip into a package
    print('Packaging files into zip with name:', baseFilename + '.zip', '...')
    zip_path = os.path.join(output_dir, baseFilename + '.zip')
    output = subprocess.check_output(                             #, shell=True
                                        [
                                            'zip', '-r', '-D',
                                            zip_path, tar_path, pdf_path
                                        ],
                                        stderr=subprocess.STDOUT,
                                        text=True
                                    ) 
    
    print('Zip packaging output:\n' + output)

except subprocess.CalledProcessError as e:
    # If the command returns a non-zero exit status, it will raise a CalledProcessError
    print('Command failed with return code:', e.returncode)
    print('Error output:\n' + e.output)#.decode('utf-8'))

### Check docker version...
#-> Available version 28.1.1 ok. 
Attempting to create Docker image with tag: openrecon_siemenshealthineersag_fetalbrainmeasurements:v1.0.0 ...
Command failed with return code: 1
Error output:
#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from OpenRecon.dockerfile
#1 transferring dockerfile: 7.26kB 0.0s done
#1 WARN: FromAsCasing: 'as' and 'FROM' keywords' casing do not match (line 7)
#1 DONE 0.1s

#2 [internal] load metadata for docker.io/nvidia/cuda:11.0.3-cudnn8-devel-ubuntu18.04
#2 ...

#3 [auth] nvidia/cuda:pull token for registry-1.docker.io
#3 DONE 0.0s

#4 [internal] load metadata for docker.io/nvidia/cuda:11.0.3-cudnn8-runtime-ubuntu18.04
#4 DONE 1.8s

#2 [internal] load metadata for docker.io/nvidia/cuda:11.0.3-cudnn8-devel-ubuntu18.04
#2 DONE 1.8s

#5 [internal] load .dockerignore
#5 transferring context: 2B done
#5 DONE 0.0s

#6 [mrd_converter 1/5] FROM docker.io/nvidia/cuda:11.0.3-cudnn8-devel

For installation on a scanner, copy the .zip file to `C:\Program Files\Siemens\Numaris\OperationalManagement\FileTransfer\incoming`. Wait a few minutes for processing and the file should be automatically removed and the app will appear under the Inline -> OpenRecon card under any sequence.